In [3]:
# This notebook takes the train and test NumPy files from the saved location, train a model on the training data, and calculate the overall
# and class-wise accuracies of the model.

In [7]:
!pip install tensorflow==2.14.0
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

  Using cached tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl.metadata (4.8 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.7-py2.py3-none-any.whl.metadata (849 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached ml_dtypes-0.2.0-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached grpcio-1.62.1-cp311-cp311-win_amd64.whl.metadata (4.2 kB)
  Using cached tensorboard-2.14.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached tensorflow_estimator-2.14.0-py2.py3-no

In [ ]:
# checks if any GPU is present for this notebook or not-

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# essentially checks the GPU type

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
# import all the necessary packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import itertools
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
%matplotlib inline

In [ ]:
# mount the google drive to load the necessary files (in the next cell).

from google.colab import drive
drive.mount('/content/Pdrive')

In [ ]:
# Load the train and test data from the NumPy files saved from the first notebook.

X_train = np.load('/content/Pdrive/MyDrive/Colab Notebooks/XTrain.npy')

X_test = np.load('/content/Pdrive/MyDrive/Colab Notebooks/XTest.npy')

y_train = np.load('/content/Pdrive/MyDrive/Colab Notebooks/yTrain.npy')

y_test = np.load('/content/Pdrive/MyDrive/Colab Notebooks/yTest.npy')

In [ ]:
# Import all the necessary things for model-training.

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Dense, Activation
from tensorflow.keras.layers import GlobalAveragePooling1D, BatchNormalization, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.regularizers import l1, l2, l1_l2 

In [ ]:
    reg2 = l2(0.01)  # L2 regularizer

    model = Sequential()
    model.add(Conv1D(filters=128,kernel_size=1, activation="relu", input_shape = (36, 1)))
    model.add(Conv1D(filters=128,kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=128, kernel_size=1, activation="relu"))
    model.add(Conv1D(filters=128, kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(Conv1D(filters=256, kernel_size=1, activation="relu"))
    model.add(MaxPooling1D(pool_size = 1))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(units=256,activation="relu", kernel_regularizer = reg2, bias_regularizer = reg2))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(units=256,activation="relu", kernel_regularizer = reg2, bias_regularizer = reg2))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(units=10, activation="softmax"))
    

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(x = X_train, y = y_train, batch_size = 128, epochs = 70)

In [ ]:
score = model.evaluate(X_test, y_test, batch_size = 128)

In [ ]:
# THE FINAL ACCURACY RESULT OF THE MODEL:

print("Accuracy: ", round(score[1]*100, 2))

In [ ]:
# y_pred = an array of all the predicted classes or labels.

y_pred = np.argmax(model.predict(X_test), axis = -1)

In [ ]:
# TO FIND THE ACCURACY OF MODEL ON EACH OF THE CLASSES 

from sklearn.metrics import confusion_matrix

cmat = confusion_matrix(y_test, y_pred)    

all_acc = cmat.diagonal()/cmat.sum(axis = 1)    # all_acc is an array containing accuracies of the model on each of the classes 

In [ ]:
all_acc

In [ ]:
#PRINT THE APPROXIMATE VALUES OF ACCURACY OF MODEL ON EACH OF THE CLASSES:

for i in range(len(all_acc)):
  
  if(i==0):
    temp = 'ALMI'
  elif(i==1):
    temp = 'AMI'
  elif(i==2):
    temp = 'ASMI'
  elif(i==3):
    temp = 'ILMI'
  elif(i==4):
    temp = 'IMI'
  elif(i==5):
    temp = 'IPLMI'
  elif(i==6):
    temp = 'IPMI'
  elif(i==7):
    temp = 'LMI'
  elif(i==8):
    temp = 'NORM'
  else:
    temp = 'PMI'
  
  print('Accuracy of the model on   ' + temp + ' = ', np.round(all_acc[i]*100, 2))